## Model Evaluation

In [1]:
%pwd

'c:\\Users\\harik\\Desktop\\All Folder\\PGDA\\Interships & Projects\\NLP\\End-to-End Workflow\\cnnClassifier\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\harik\\Desktop\\All Folder\\PGDA\\Interships & Projects\\NLP\\End-to-End Workflow\\cnnClassifier'

In [5]:
import tensorflow as tf

In [6]:
# To do the evaluation we need to load the model

In [7]:
model= tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
# Now we need to write the entity

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [16]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories, save_json
import tensorflow as tf

In [11]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])


    def get_validation_config(self) -> EvaluationConfig:
          eval_config= EvaluationConfig(
                path_of_model= "artifacts/training/model.h5",
                training_data= "artifacts/data_ingestion/PetImages",
                all_params= self.params,
                params_image_size= self.params.IMAGE_SIZE,
                params_batch_size= self.params.BATCH_SIZE
          )
          return eval_config

In [12]:
# Now we make components

In [13]:
from urllib.parse import urlparse

In [18]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config= config

    def _valid_generator(self):

        datagenerator_kwargs= dict(
            rescale= 1./255,        # Here we doing the basic rescaling, which is making the images from 0-1
            validation_split= 0.20
        )

        dataflow_kwargs= dict(
            target_size= self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation= "bilinear"
        ) 
          # Here when we resize the image to target size, the images will be shrinked
          # therefore new pixels are created and interpolation specify which method
          # should be used to colour the new images, here we use bilinear method
          # which means giving a blende ( averaging ) colur from the nearest pixels

        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator= valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= "validation",
            shuffle= False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model= self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score= model.evaluate(self.valid_generator)


        """ 
            Here in ._valid_generator() we are getting subset of validation from the training data as in self.valid_generator
            we have given the directory and the subset, and we have given some dataflow_kwargs to alter the data.
            now with self.score in the variable score model will be evaluated with the valid generator data, and in the next code
            its been saved in a json.

            Basically here it acts like a test data, in which we are testing our model and the loass and accuracy of the same
            is saved in the json file.
        
        """

    
    def save_score(self):
        scores= {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path= Path("scores.json"), data=scores)


In [19]:
try:
    config= ConfigurationManager()
    val_config= config.get_validation_config()
    evaluation= Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e

[2023-10-13 21:46:29,368: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2023-10-13 21:46:29,368: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-10-13 21:46:29,376: INFO: common: created directory at: artifacts]


Found 78 images belonging to 2 classes.
5/5 [==============================] - 14s 3s/step - loss: 9.0317 - accuracy: 0.5000
[2023-10-13 21:46:44,127: INFO: common: json saved at scores.json]
